# Versuche für Filter-Stage
Wie Oude Elberink et al. 2013 und auch in Oude Elberink und Kh. 2015 und in Kononen et al. 2024

In [75]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os
import json
import pyvista as pv

from interessant import * # Bei Änderungen Kernel neu starten

- Erster Durchgang `thresh = 5` und `majority_tresh  = 0.3`
- Zweiter Durchgang `thresh = 8` und `majority_tresh  = 0.5`

Beobachtungen
- "Weiche": Eine graue Kachel im Gleisbereich wenn `thresh < 6` 
- Probleme neben Bahnsteig (graues Gleis): "Ende", "Unterirdischer Bhf", "Bahnsteig"
    - Evtl Kacheln kleiner als 1 m versuchen?
    - Betrifft vor allem Ecklagen, vielleicht geht es auch so?
- Bahnübergang natürlich blau
- Falsche rote Linien
    - Eine rote Linie im "Feldweg"
    - rote Linie "Betondeckel"
- Blaue Kacheln im Gleisbereich
    - "Weiche A" mehrere bei majority_thresh 0.3, OK bei 0.5
    - 'Zug run 14 A' in run24: eine blaue Kachel im Gleisbereich bei majority_thres 0.3, OK bei 0.5
    - "Weiche B" mehrere, wo wegen Weichenherz oder Fangschienen viele Punkte erhöht sind
    - "Viele Gleise": mit zunehmender Entfernung immer mehr blaue Kacheln, 0.5 viel besser, aber noch immer blaue Lücken
    - "OLA gleiche Höhe wie Gleis" eine Kachel (auch bei 0.5)
- majority_tresh  
    - mit 0.3 'Gleis hohe Intensität' und 'Y' fast alles Gleis blau (!), nur vereinzelt wegen Intensität ausgefiltert. 
    - 0.5 deutlich besser (fast alles orange)
    - Bei "Gleis weit abseits" wird das äußerste Gleis bei majority_thresh 0.5 nicht mehr wirklich erkannt (meist blau, schwarz, orange, ganz wenige rote Punkte)
    - 0.5 viel besser (viel rot/orange, auch schwarz, kaum blau)
    - 0.3 mehrere blau in "Wände", nur eine mit 0.5
- Idee: Filter auf Intensität nur bei der Auswahl der Saatpunkte, aber pca und template matching einschließlich der entsprechenden Punkte
- Grüne Kacheln im Gleisbereich:
    - In "Bahnsteig" ist am Rand gerade noch ein Teil von einem Busch in der Kachel, wenige Punkte, aber die sind wahrscheinlich zu hoch um das tiefer liegende Gleis in der anderen Ecke zu finden?
    - "4 Gleise" auch wegen Busch
- Intensity Filter positiv
    - "Kante im Graben" erfolgreich mit Intensität ausgefiltert
    - Viele in "Einfach"
- Intensity Filter falsch
    - "OLA auf gleicher Höhe wie Gleis": unteres Gleis viel orange (hohe Intensität)
    - 'Gleis hohe Intensität' (bei majority_thresh 0.5)
    - "Weiche B" und "Fangschiene Tunnel 2" viele Punkte an Gleisinnenkante orange, auch wenn viele rote übrig bleiben
    - "Zaun" (idx 17) in run14: Gleis viel orange
    - "Gleis weit abseits" in run14: äußertes Gleis orange
- "Kabeltöpfe" 
    - Böschung mit falsch positiv, durch Intensity threshold herausgefiltert
    - eine graue Kachel im Gleisbereich bei majority_thres 0.3, OK bei 0.5
- Rote (nicht lineare) Kästen in "Anfang Weiche"
- "4 Gleise": weit entferntes Gleis z.T. grün (viel occlusion)
- Steigung eher geringes Problem!
    - Bei 4° Steigung wird oberer Teil des Schienenkopfs noch durchgehend erkannt
    - unterer Teil der Schiene wird nur gestuft erfasst
- Zeiten bei Kachel "Einfach": Sehr schnell und dafür erstaunlich gut!
    - Datei lesen 1.3 s
    - pcd (open3d) erstellen 0.1 s
    - voxel erstellen 0.1 s
    - voxel_dict erstellen 5.8 s
    - constraints 0.7 s
    - nicht mitgezählt 4.9 s beim import (nur einmal nötig)
- Oude E. & K. nehmen die 98. Percentile für das top des Schienenkopfes, vermutlich wegen Noise. Bei mir gehen dann alle Rail-Punkte verloren, wo das Gleis nur in der Ecke des Voxels vorkommt. In fast allen Fällen ist Problem bei 99.9 Percentile behoben
- Voxelgröße 25/30 (ca. 8.33) 
    - löst das Bahnsteigproblem und weniger Probleme mit Büschen
    - Unterirdischer Bhf (idx 20) in run14 noch immer Bahnsteigproblem, aber nur kleine Ecken
    - Weichenherz in Weiche B aber blau
    - Y bei run14 Gleis manchmal blau (seltener bei voxelgröße 1)
    - bei weit abgelegenen Gleisen wird viel schwarz mit minimum_points=100, besser 50
- Noise Filter: 
    - in "viele Gleise" bei abseits gelegenem Gleis gehen viele wichtige Punkte verloren
    - Ansonsten i.d.R. eher positiver Effekt

    

In [76]:
# Zeiten, Prozent: 72.5 % für Erstellung von voxel_dict; 16 % zum Lesen der Datei
a = np.array([1.3, 0.1, 0.1, 5.8, 0.7])
total = np.sum(a)
print(100*a/total)

[16.25  1.25  1.25 72.5   8.75]


In [77]:
print(len(interessant.keys()))
interessant.keys()

48


dict_keys(['Einfach', 'Diagonal', 'Weiche', 'Ende', 'Bahnübergang', 'Bahnübergang 2', 'Zug run 14 A', 'Zug run 14 B', 'Zug run 24 A', 'Zug run 24 B', 'Gebäude', 'Gleis hohe Intensität', 'Gleis hohe Intensität 2', 'Feldweg', 'Feld', 'Weiche A', 'Weiche B', 'Zaun', 'Straße', 'Kante in Graben', 'Unterirdischer Bhf', 'Fangschiene Tunnel', 'Fangschiene Tunnel 2', 'Gleis weit abseits', 'Komische Linie', 'Kabeltöpfe', 'Güterzug', 'Güterzug Ende', 'Betondeckel', 'Bahnsteig', 'Bahnsteig Ende', 'Ding neben Gleis', 'Wände', 'Viele Gleise', 'Anfang Weiche', 'OLA gleiche Höhe wie Gleis', '4 Gleise', 'Y', 'Weiche C', 'Weiche D', 'Für template', 'Rand', 'Extrem viele Punkte run24', 'Viele Gleise 2', 'Kreuzung linker Teil', 'Kreuzung rechter Teil', 'Weiche abseits', 'Drei'])

In [78]:
# for i, key in enumerate(interessant.keys()):
#     print(i, key)

In [79]:
run = run24
#run = run14
# filename = interessant['OLA gleiche Höhe wie Gleis']

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A (in run24 Achszähler): 6; 
# Viele Gleise: 33; Anfang Weiche: 34, OLA gleiche H: 35; Y: 37
key = list(interessant.keys())[0] 
filename = interessant[key]
print(key, filename)


#filename = "4479050_5352975.copc.laz"


filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

Einfach 4473900_5335875.copc.laz


In [80]:
# Alternativ: Steigung

# filename = os.path.join(basedir, "steigung", steigung[3])

# print(filename)
# os.path.exists(filename)

In [81]:
thresh = 20  # z.B. 5 oder 8
majority_tresh  = 0.7# 0.7  # 0.7 0.5 # Erster Durchgang 0.3, bei "Gleis hohe Intensität" gibt 0.5 ein viel besseres Ergebnis
ground_percentile = 10 # 10 # 10 in Oude E.

voxel_size = 1.0

voxel_size = 25 / 30
print("Voxel size:", voxel_size)

minimum_points = 50 # Erste Versuche mit 100, aber viel schwarz bei abseits liegenden Gleisen. 50 ist besser.

with_normals = False

Voxel size: 0.8333333333333334


In [82]:
import subprocess
# subprocess.Popen(["pyvistaviewer", filename])

In [83]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]
points.dtype # Column names and types

dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('Synthetic', 'u1'), ('KeyPoint', 'u1'), ('Withheld', 'u1'), ('Overlap', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('ScanChannel', 'u1'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])

In [84]:
metadata = pipeline.metadata['metadata']
metadata['readers.copc']['maxx']

4473925

In [85]:
def pipeline_bbox(pipeline):
    """Return 2D bounding box of the point cloud"""
    metadata = pipeline.metadata['metadata']
    key = None
    for k in metadata.keys():
        if k.startswith('readers'):
            key = k
    if not key:
        raise ValueError("No readers found")
    maxx = metadata[key]['maxx']
    minx = metadata[key]['minx']
    maxy = metadata[key]['maxy']
    miny = metadata[key]['miny']

    return minx, miny, maxx, maxy

In [86]:
pipeline_bbox(pipeline)

(4473900, 5335875, 4473925, 5335900)

In [87]:
points['Classification'] = 0 # Unclassified

In [88]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()
rgb = np.vstack((points['Red'], points['Green'], points['Blue'])).transpose() / 65535.0

intensity = points['Intensity']
intensity_normalized = (intensity - intensity.min()) / (intensity.max() - intensity.min())
colormap = plt.get_cmap("viridis")
intensity_colors = colormap(intensity_normalized)
intensity_colors = intensity_colors[:, :3]

# Offset entfernen (aber gerundet, damit Kachelgrenzen ganze Zahlen bleiben)
offset = xyz.mean(axis=0).round() 
xyz -= offset

In [89]:
offset

array([4.473913e+06, 5.335888e+06, 5.170000e+02])

In [90]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
# pcd.colors = o3d.utility.Vector3dVector(intensity_colors)
# pcd.colors = o3d.utility.Vector3dVector(rgb)

In [91]:
# o3d.visualization.draw_geometries([pcd])

# Teile in säulenartige Voxel, um darin z zu untersuchen

In [92]:
maxp = xyz.max(axis=0)
minp = xyz.min(axis=0)
maxp, minp

(array([11.99997543, 11.99998693, 13.12408   ]),
 array([-13.00002457, -13.00001307,  -3.99652   ]))

In [93]:
my_classes = {
    "Unclassified": 0,
    "High Points": 13,   
    "No clearance": 14,
    "Missing Rail": 15,
    "Low Points": 16,
    "Rail": 20,
}

color_map = {
    13: [0.3, 0.3, 0.3],
    14: [0.5, 0.5, 0.5], 
    15: [0, 0, 1],
    16: [0, 1, 0],
    20: [1, 0, 0],
}

In [94]:
voxels = xyz.copy()
voxels[:, :2] = ((xyz[:, :2] - minp[:2]) // voxel_size).astype(int)

In [95]:
# Anzahl der Voxel checken
np.ceil((maxp[:2] - minp[:2]) / voxel_size).astype(int)

array([30, 30])

In [96]:
voxels  

array([[15.     , 27.     , 11.86328],
       [17.     , 23.     , 12.22228],
       [15.     , 26.     , 11.89178],
       ...,
       [25.     , 29.     ,  1.27228],
       [21.     , 27.     ,  0.67448],
       [23.     , 21.     , -1.37662]])

In [97]:
len(voxels) 

1803127

Die folgende Zelle benötigt bei Kachel "Einfach" 5.8 s.

In [98]:
from collections import defaultdict
voxel_dict = defaultdict(list)
index_dict = defaultdict(list)

# Füllen des Dictionaries
for idx, (point, voxel) in enumerate(zip(xyz, voxels)):
    voxel_key = tuple(voxel[:2])
    voxel_dict[voxel_key].append(point[2])
    index_dict[voxel_key].append(idx)

In [99]:
# Viewsettings mit strg + c kopieren und hier einfügen

viewsettings = '''
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 11.999975427985191, 11.99998692702502, 13.124079998226534 ],
			"boundingbox_min" : [ -13.000024572014809, -13.00001307297498, -3.9965200017734333 ],
			"field_of_view" : 60.0,
			"front" : [ -0.20468464372193082, -0.82045900926496551, 0.53380821531742795 ],
			"lookat" : [ -2.1145501200370735, -2.6052610037108783, 1.4494799802055294 ],
			"up" : [ 0.19010212482081987, 0.50164960558000959, 0.84392467398461002 ],
			"zoom" : 0.55999999999999983
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

'''

viewsettings = json.loads(viewsettings)

front = viewsettings["trajectory"][0]["front"]
lookat = viewsettings["trajectory"][0]["lookat"]
up = viewsettings["trajectory"][0]["up"]
zoom = viewsettings["trajectory"][0]["zoom"]

- Folgende Zelle benötigt 0.7s
- `thresh` <= 5 gibt bei Weiche eine graue Kachel im Gleis

In [100]:

for key, z_values in voxel_dict.items():
    
    # Threshold on number of points in voxel
    if len(z_values) < minimum_points:
        continue

    indices = np.array(index_dict[key])
    z_values = np.array(z_values)
    ground_level = np.percentile(z_values, ground_percentile) # 10% Percentile in Oude E.
    # Check that there are almost no points 0.5 to 4.5 m above the ground
    # But allow for some noise
    count = ((z_values > ground_level + 0.5) & (z_values < ground_level + 4.5)).sum()

    if count <= thresh:
        # Look for points within 0.5 m above ground and get 98% percentile ODER 99.5
        mask = (z_values > ground_level) & (z_values < ground_level + 0.5)
        try:
            candidates_top = np.percentile(z_values[mask], 99.5) # 99.5 ##################################################################################
        except IndexError:
            # Fails if there are no points in the masked array
            points['Classification'][indices] = my_classes["Missing Rail"]
            continue

        # Oude Elberink require the height difference > 0.1 m
        # And mark only the points 10 cm below the top as rail point candidates
        if candidates_top - ground_level > 0.1:
            mask = (z_values > candidates_top - 0.1) & (z_values < candidates_top + 0.005)

            # Also make sure these are only a minority of the points (otherwise it's a slope)
            if mask.sum() < majority_tresh * len(z_values):  # z.B. 0.3
                points['Classification'][indices[mask]] = my_classes["Rail"]
                points['Classification'][indices[~mask]] = my_classes["Low Points"]
            else:
                points['Classification'][indices] = my_classes["High Points"]     #["Missing Rail"]  ################################


        else:
            # There are no points above ground 
            points['Classification'][indices] = my_classes["Missing Rail"]


        # Class for high points
        mask = (z_values >= ground_level + 4.5)
        points['Classification'][indices[mask]] = my_classes["High Points"]


    else:
        # Discard voxel
        # Set color of all points in voxel to grey
        points['Classification'][indices] = my_classes["No clearance"]




Oude E. & K. nehmen die 98. Percentile, vermutlich wegen Noise. Zum Teil gehen mir Rail-Punkte verloren, wenn das Gleis nur in der Ecke des Voxels vorkommt. In vielen Fällen ist Problem bei 99.9 Percentile behoben

In [101]:
class_colors = np.zeros_like(rgb)
for class_value, color in color_map.items():
    class_colors[points['Classification'] == class_value] = color

In [102]:
# Optional Schwellenwert für Intensität
intensity_threshold = 14500

mask = (points["Classification"] == my_classes["Rail"]) & (points["Intensity"] > intensity_threshold)
class_colors[mask] = [0.9, 0.5, 0]

In [103]:
# Normals für Visualisierung
if with_normals:
    nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())  
    print(nn_distance)  
    radius_normals=nn_distance*10 #10  # 4
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=50), fast_normal_computation=True) 

In [104]:
pcd.colors = o3d.utility.Vector3dVector(class_colors)
o3d.visualization.draw_geometries([pcd], front=front, lookat=lookat, up=up, zoom=zoom)

In [105]:
# out_pipeline = pdal.Writer("test.laz").pipeline(points)
# out_pipeline.execute()

In [106]:
# out_pipeline.toJSON()

In [107]:
# ggf. hier mit exception stoppen
raise ValueError("Stop here")

ValueError: Stop here

## GIF

In [ ]:
from sonstiges.cloudgify import cloudgify

In [ ]:
pvpcd = pv.PolyData(xyz)
pvpcd['class'] = class_colors
cloudgify(pvpcd, scalars='class', rgb=True, background_color="#ffffff", style='surface', eye_dome_lighting=True, point_size=1.5)

Widget(value='<iframe src="http://localhost:43391/index.html?ui=P_0x7fb8b67eff10_0&reconnect=auto" class="pyvi…

## Nur mit Kandidatenpunkten weiter

In [ ]:
candidates = points[points["Classification"] == my_classes["Rail"]]

In [ ]:
len(candidates)

92692

In [ ]:
noise_filter = pdal.Filter("filters.outlier", method="statistical", mean_k=10, multiplier=2.0).pipeline(candidates) # | pdal.Filter("filters.range", limits="Classification![7:7]")
print(noise_filter.toJSON())
noise_filter.execute() # Wirft exception wenn keine candidates
candidates = noise_filter.arrays[0]
candidates.shape 

[{"type": "filters.outlier", "method": "statistical", "mean_k": 10, "multiplier": 2.0, "tag": "filters_outlier1"}]


(92692,)

In [ ]:
noise_colors = np.zeros((candidates.shape[0], 3), dtype=np.float64) 

In [ ]:
noise_colors[:,2] = 1
noise_colors

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [ ]:
mask = candidates["Intensity"] > intensity_threshold
noise_colors[mask] = [0.5, 0.5, 0.5]

In [ ]:
noise_colors[candidates['Classification'] == 7] = [1, 0, 0]

In [ ]:
xyz = np.vstack((candidates['X'], candidates['Y'], candidates['Z'])).transpose()
xyz -= offset

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(noise_colors)

# nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())  
# print(nn_distance)  
# radius_normals=nn_distance*4 #10  # 4
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=50), fast_normal_computation=True) 



o3d.visualization.draw_geometries([pcd], front=front, lookat=lookat, up=up, zoom=zoom)

In [ ]:
mask = (candidates['Classification'] != 7) & (candidates['Intensity'] < intensity_threshold)
out_pipeline = pdal.Writer("result_voxelfilter.laz").pipeline(candidates[mask])
out_pipeline.execute()

49162

In [ ]:
filename 

'/media/riannek/minimax/gleis/2024-08-13/01/run24/01/4473900_5335875.copc.laz'

In [ ]:
pipeline = pdal.Pipeline([pdal.Reader(filename, count=0)])
pipeline.execute()

0

In [ ]:
pipeline.metadata['metadata']['readers.copc']['srs']['json']['id']['code']

5684